<a href="https://colab.research.google.com/github/NiangZd/API-ATV-GQS/blob/main/Pipeline_de_Classifica%C3%A7%C3%A3o_MNIST_(dividido_em_c%C3%A9lulas).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # Pipeline de Classificação MNIST (Dividido em Células)
# **Etapas:**
# * a) Avaliação com `cross_val_predict` (em um subset rápido)
# * b) Ranqueamento por Acurácia CV (Top 3)
# * c) Avaliação em um conjunto de teste inicial (treinado no subset rápido)
# * d) Ranqueamento por estabilidade (Diferença CV-Teste) (Top 3)
# * e) `RandomizedSearchCV` nos Top 3 de estabilidade (usando subset rápido)
# * f) Avaliação dos modelos otimizados em um NOVO conjunto de teste (treinado no set completo)
# * g) Exportação do melhor modelo final com `joblib`.

# ## Célula 1: Importações
# Importa todas as bibliotecas necessárias.

import numpy as np
import pandas as pd
import joblib
import warnings
import time
from sklearn.datasets import fetch_openml
from sklearn.model_selection import cross_val_predict, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from scipy.stats import uniform, randint

from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [ ]:
# ## Célula 2: Configuração Inicial
# Define as constantes globais, SEEDs e configurações do pipeline.

warnings.filterwarnings("ignore")

# Constantes de configuração
SEED = 42
N_ITER_RANDOM = 10  # N de combinações para o RandomizedSearch
N_FOLDS_CV = 3      # N de folds para a validação cruzada
FAST_SUBSET_SIZE = 10000 # Tamanho do subset para rodar rápido
MODEL_FILENAME = 'best_mnist_model.joblib'

np.random.seed(SEED)
print(f"Iniciando pipeline de classificação MNIST...")
print(f"Configurações: SEED={SEED}, N_ITER_RANDOM={N_ITER_RANDOM}, N_FOLDS_CV={N_FOLDS_CV}\n")

Iniciando pipeline de classificação MNIST...
Configurações: SEED=42, N_ITER_RANDOM=10, N_FOLDS_CV=3



In [ ]:
# ## Célula 3: Carga e Pré-processamento dos Dados
# Carrega o dataset MNIST e aplica o `StandardScaler`.

print("Carregando dataset MNIST (pode levar um momento)...")
# Carrega 70.000 imagens (28x28 = 784 features)
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
X = mnist.data.astype('float32')
y = mnist.target.astype(np.uint8)

print("Pré-processando os dados...")
# Normalização (StandardScaler)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Dados carregados e normalizados.")

Carregando dataset MNIST (pode levar um momento)...
Pré-processando os dados...
Dados carregados e normalizados.


In [ ]:
# ## Célula 4: Estratégia de Divisão (Splitting) Robusta
# Divide os dados em múltiplos conjuntos

print("Dividindo os dados...")

# Split A: Separa o conjunto de "Teste Final" (10k).
X_train_val, X_test_final, y_train_val, y_test_final = train_test_split(
    X_scaled, y, test_size=10000, random_state=SEED, stratify=y
)

# X_train_val: 60k amostras
# X_test_final: 10k amostras

# Split B: Separa o "Teste Inicial" (10k) do conjunto de treino/validação.
X_train_full, X_test_initial, y_train_full, y_test_initial = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=SEED, stratify=y_train_val
)

# X_train_full: 50k amostras
# X_test_initial: 10k amostras

# Split C: Cria o "Subset Rápido" (10k).
# Usamos 'train_size' para pegar uma fração de X_train_full
_, X_train_fast, _, y_train_fast = train_test_split(
    X_train_full, y_train_full, train_size=FAST_SUBSET_SIZE, random_state=SEED, stratify=y_train_full
)
# X_train_fast: 10k amostras (usado para CV rápido e RandomSearch)

print(f"Divisão de dados concluída:")
print(f" - Subset de Treino/CV Rápido (a,c,e): {X_train_fast.shape[0]} amostras")
print(f" - Teste Inicial (c):                 {X_test_initial.shape[0]} amostras")
print(f" - Treino Completo (f):               {X_train_val.shape[0]} amostras")
print(f" - Teste Final (f):                   {X_test_final.shape[0]} amostras")

Dividindo os dados...
Divisão de dados concluída:
 - Subset de Treino/CV Rápido (a,c,e): 40000 amostras
 - Teste Inicial (c):                 10000 amostras
 - Treino Completo (f):               60000 amostras
 - Teste Final (f):                   10000 amostras


In [ ]:
# ## Célula 5: Definição dos Modelos Base
# Cria o dicionário `models` com os 7 classificadores usando seus hiperparâmetros default.

models = {
    "NaiveBayes": GaussianNB(),
    "MLP": MLPClassifier(random_state=SEED, max_iter=300),
    "KNN": KNeighborsClassifier(n_jobs=-1),
    "Regressão Logística": LogisticRegression(random_state=SEED, max_iter=200, n_jobs=-1),
    "SGD": SGDClassifier(random_state=SEED, max_iter=200, n_jobs=-1),
    "XGBoost": XGBClassifier(random_state=SEED, use_label_encoder=False, eval_metric='mlogloss', n_jobs=-1),
    "Árvore de Decisão": DecisionTreeClassifier(random_state=SEED)
}
print(f"{len(models)} modelos base definidos.")

7 modelos base definidos.


In [ ]:
# ## Célula 6: Definição das Distribuições de Hiperparâmetros
# Cria o dicionário `param_distributions` para a busca aleatória

param_distributions = {
    "NaiveBayes": {
        'var_smoothing': uniform(1e-10, 1e-8)
    },
    "MLP": {
        'hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'activation': ['relu', 'tanh'],
        'alpha': uniform(1e-5, 1e-3),
        'learning_rate_init': uniform(0.001, 0.01)
    },
    "KNN": {
        'n_neighbors': randint(3, 9),
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    },
    "Regressão Logística": {
        'C': uniform(0.1, 10),
        'solver': ['saga'],
        'penalty': ['l1', 'l2']
    },
    "SGD": {
        'loss': ['hinge', 'log_loss'],
        'penalty': ['l2', 'l1', 'elasticnet'],
        'alpha': uniform(1e-5, 1e-3)
    },
    "XGBoost": {
        'n_estimators': randint(50, 200),
        'learning_rate': uniform(0.01, 0.2),
        'max_depth': randint(3, 7)
    },
    "Árvore de Decisão": {
        'criterion': ['gini', 'entropy'],
        'max_depth': randint(5, 20),
        'min_samples_split': randint(2, 20)
    }
}
print("Distribuições de hiperparâmetros definidas.")

Distribuições de hiperparâmetros definidas.


In [ ]:
# ## Célula 7: Inicialização de Variáveis
# Prepara os dicionários para armazenar resultados e marca o tempo de início.

# Dicionários para guardar resultados
cv_results = {}
test_results = {}
final_results = {}
best_tuned_models = {}

start_time = time.time()
print("Variáveis de resultado inicializadas.")

Variáveis de resultado inicializadas.


In [ ]:
# ## Célula 8: Etapa (a) - Validação Cruzada (CV)
# Avalia os 7 modelos default usando `cross_val_predict` no *subset rápido* (10k amostras).

print("\n--- Etapa (a): Iniciando Validação Cruzada (CV) ---")
print(f"(Usando {X_train_fast.shape[0]} amostras e {N_FOLDS_CV} folds)")

for name, model in models.items():
    t0 = time.time()
    print(f"Avaliando {name}...")
    y_pred_cv = cross_val_predict(model, X_train_fast, y_train_fast, cv=N_FOLDS_CV, n_jobs=-1)
    acc_cv = accuracy_score(y_train_fast, y_pred_cv)
    cv_results[name] = acc_cv
    print(f"  -> {name} | Acurácia CV: {acc_cv:.4f} (levou {time.time()-t0:.2f}s)")


--- Etapa (a): Iniciando Validação Cruzada (CV) ---
(Usando 40000 amostras e 3 folds)
Avaliando NaiveBayes...
  -> NaiveBayes | Acurácia CV: 0.5424 (levou 6.43s)
Avaliando MLP...
  -> MLP | Acurácia CV: 0.9646 (levou 59.63s)
Avaliando KNN...
  -> KNN | Acurácia CV: 0.9324 (levou 70.59s)
Avaliando Regressão Logística...
  -> Regressão Logística | Acurácia CV: 0.8978 (levou 71.31s)
Avaliando SGD...
  -> SGD | Acurácia CV: 0.9098 (levou 133.29s)
Avaliando XGBoost...
  -> XGBoost | Acurácia CV: 0.9686 (levou 799.42s)
Avaliando Árvore de Decisão...
  -> Árvore de Decisão | Acurácia CV: 0.8489 (levou 29.57s)


In [ ]:
# ## Célula 9: Etapa (c) - Avaliação no Teste Inicial
# Treina os modelos default no *subset rápido* (10k) e avalia no *teste inicial* (10k).

print("\n--- Etapa (c): Avaliando no Teste Inicial ---")
print(f"(Treinando em {X_train_fast.shape[0]}, testando em {X_test_initial.shape[0]})")

for name, model in models.items():
    t0 = time.time()
    print(f"Treinando e testando {name}...")
    model.fit(X_train_fast, y_train_fast) # Treina no subset rápido
    y_pred_test = model.predict(X_test_initial) # Testa no teste inicial
    acc_test = accuracy_score(y_test_initial, y_pred_test)
    test_results[name] = acc_test
    print(f"  -> {name} | Acurácia Teste Inicial: {acc_test:.4f} (levou {time.time()-t0:.2f}s)")


--- Etapa (c): Avaliando no Teste Inicial ---
(Treinando em 40000, testando em 10000)
Treinando e testando NaiveBayes...
  -> NaiveBayes | Acurácia Teste Inicial: 0.5476 (levou 0.94s)
Treinando e testando MLP...
  -> MLP | Acurácia Teste Inicial: 0.9706 (levou 40.61s)
Treinando e testando KNN...
  -> KNN | Acurácia Teste Inicial: 0.9452 (levou 27.32s)
Treinando e testando Regressão Logística...
  -> Regressão Logística | Acurácia Teste Inicial: 0.9087 (levou 40.72s)
Treinando e testando SGD...
  -> SGD | Acurácia Teste Inicial: 0.9164 (levou 63.25s)
Treinando e testando XGBoost...
  -> XGBoost | Acurácia Teste Inicial: 0.9769 (levou 390.39s)
Treinando e testando Árvore de Decisão...
  -> Árvore de Decisão | Acurácia Teste Inicial: 0.8667 (levou 16.21s)


In [ ]:
# ## Célula 10: Etapa (b) - Ranqueamento por Acurácia CV (Top 3)

results_df = pd.DataFrame({
    'Acuracia_CV': cv_results,
    'Acuracia_Teste': test_results
})
results_df['Diferenca_Abs'] = abs(results_df['Acuracia_CV'] - results_df['Acuracia_Teste'])

print("\nDataFrame de Resultados (CV vs Teste Inicial):")
print(results_df.round(4))

print("\n--- Etapa (b): Ranqueamento por Acurácia CV ---")
results_cv_ranked = results_df.sort_values(by='Acuracia_CV', ascending=False)
print(results_cv_ranked[['Acuracia_CV']])
top_3_cv_names = results_cv_ranked.index[:3].tolist()
print(f"\nTop 3 (Acurácia CV): {', '.join(top_3_cv_names)}")


DataFrame de Resultados (CV vs Teste Inicial):
                     Acuracia_CV  Acuracia_Teste  Diferenca_Abs
NaiveBayes                0.5424          0.5476         0.0052
MLP                       0.9646          0.9706         0.0060
KNN                       0.9324          0.9452         0.0128
Regressão Logística       0.8978          0.9087         0.0109
SGD                       0.9098          0.9164         0.0066
XGBoost                   0.9686          0.9769         0.0083
Árvore de Decisão         0.8489          0.8667         0.0178

--- Etapa (b): Ranqueamento por Acurácia CV ---
                     Acuracia_CV
XGBoost                 0.968600
MLP                     0.964550
KNN                     0.932425
SGD                     0.909775
Regressão Logística     0.897800
Árvore de Decisão       0.848875
NaiveBayes              0.542375

Top 3 (Acurácia CV): XGBoost, MLP, KNN


In [ ]:
# ## Célula 11: Ranqueamento por Estabilidade e Análise
# Classifica os modelos pela menor `Diferenca_Abs` (CV vs Teste) e identifica o Top 3 (modelos mais estáveis),
# e analisa a diferença entre os Top 3 de acurácia e estabilidade.

print("\n--- Etapa (d): Ranqueamento por Estabilidade (Menor Diferença CV-Teste) ---")
results_diff_ranked = results_df.sort_values(by='Diferenca_Abs', ascending=True)
print(results_diff_ranked[['Acuracia_CV', 'Acuracia_Teste', 'Diferenca_Abs']].round(4))
top_3_stability_names = results_diff_ranked.index[:3].tolist()
print(f"\nTop 3 (Estabilidade): {', '.join(top_3_stability_names)}")

# Análise (d)
print("\n--- Análise (d): Melhores CV vs. Mais Estáveis ---")
coincidem = set(top_3_cv_names) == set(top_3_stability_names)
print(f"Top 3 Acurácia:   {top_3_cv_names}")
print(f"Top 3 Estabilidade: {top_3_stability_names}")
print(f"Os conjuntos {'COINCIDEM' if coincidem else 'NÃO COINCIDEM'}.")
print("Frequentemente, os modelos mais precisos (ex: XGB, MLP) podem ter mais overfitting")
print("com parâmetros default, mostrando uma diferença maior, enquanto modelos mais simples")
print("(ex: Regressão Logística, NaiveBayes) são mais estáveis.")


--- Etapa (d): Ranqueamento por Estabilidade (Menor Diferença CV-Teste) ---
                     Acuracia_CV  Acuracia_Teste  Diferenca_Abs
NaiveBayes                0.5424          0.5476         0.0052
MLP                       0.9646          0.9706         0.0060
SGD                       0.9098          0.9164         0.0066
XGBoost                   0.9686          0.9769         0.0083
Regressão Logística       0.8978          0.9087         0.0109
KNN                       0.9324          0.9452         0.0128
Árvore de Decisão         0.8489          0.8667         0.0178

Top 3 (Estabilidade): NaiveBayes, MLP, SGD

--- Análise (d): Melhores CV vs. Mais Estáveis ---
Top 3 Acurácia:   ['XGBoost', 'MLP', 'KNN']
Top 3 Estabilidade: ['NaiveBayes', 'MLP', 'SGD']
Os conjuntos NÃO COINCIDEM.
Frequentemente, os modelos mais precisos (ex: XGB, MLP) podem ter mais overfitting
com parâmetros default, mostrando uma diferença maior, enquanto modelos mais simples
(ex: Regressão Logística, 

In [ ]:
# ## Célula 12: - Randomized Search (Otimização)
# Executa o `RandomizedSearchCV` (com `N_ITER_RANDOM` iterações) nos **Top 3 de estabilidade**,
# usando o *subset rápido* (10k) para agilidade.

print(f"\n--- Etapa (e): Randomized Search (N={N_ITER_RANDOM}) nos Top 3 de Estabilidade ---")
print(f"(Usando {X_train_fast.shape[0]} amostras para a busca)")

models_to_tune = {name: models[name] for name in top_3_stability_names}

for name, model in models_to_tune.items():
    t0 = time.time()
    print(f"\nOtimizando {name}...")

    if name not in param_distributions:
        print(f"  -> Sem distribuição de parâmetros definida para {name}. Pulando.")
        best_tuned_models[name] = model
        continue

    rand_search = RandomizedSearchCV(
        model,
        param_distributions[name],
        n_iter=N_ITER_RANDOM,
        cv=N_FOLDS_CV,
        scoring='accuracy',
        random_state=SEED,
        n_jobs=-1,
        verbose=0
    )

    # Ajusta a busca no subset rápido
    rand_search.fit(X_train_fast, y_train_fast)

    best_tuned_models[name] = rand_search.best_estimator_ # Guarda o melhor modelo

    print(f"  -> {name} Concluído (levou {time.time()-t0:.2f}s)")
    print(f"  -> Melhor Acurácia CV (na busca): {rand_search.best_score_:.4f}")
    print(f"  -> Melhores Parâmetros: {rand_search.best_params_}")


--- Etapa (e): Randomized Search (N=10) nos Top 3 de Estabilidade ---
(Usando 40000 amostras para a busca)

Otimizando NaiveBayes...
  -> NaiveBayes Concluído (levou 32.16s)
  -> Melhor Acurácia CV (na busca): 0.5516
  -> Melhores Parâmetros: {'var_smoothing': np.float64(9.607143064099161e-09)}

Otimizando MLP...
  -> MLP Concluído (levou 506.85s)
  -> Melhor Acurácia CV (na busca): 0.9671
  -> Melhores Parâmetros: {'activation': 'relu', 'alpha': np.float64(0.000606850157946487), 'hidden_layer_sizes': (100,), 'learning_rate_init': np.float64(0.0025599452033620265)}

Otimizando SGD...
  -> SGD Concluído (levou 5569.68s)
  -> Melhor Acurácia CV (na busca): 0.9088
  -> Melhores Parâmetros: {'alpha': np.float64(0.00016601864044243653), 'loss': 'hinge', 'penalty': 'elasticnet'}


In [ ]:
# ## Célula 13:- Avaliação Final (no Teste Final)
# Re-treina os 3 modelos otimizados no *conjunto de treino completo* (60k)
# e avalia no *conjunto de teste final* (10k).

print("\n--- Etapa (f): Avaliação Final no Conjunto de Teste Final (10k) ---")
print(f"(Re-treinando os 3 melhores modelos em {X_train_val.shape[0]} amostras)")

for name, model in best_tuned_models.items():
    t0 = time.time()
    print(f"Re-treinando e avaliando {name} otimizado...")

    # Re-treina o modelo (com os melhores params) no conjunto de treino COMPLETO (60k)
    model.fit(X_train_val, y_train_val)

    # Avalia no conjunto de teste FINAL (10k)
    y_pred_final = model.predict(X_test_final)
    acc_final = accuracy_score(y_test_final, y_pred_final)
    final_results[name] = acc_final

    print(f"  -> {name} | Acurácia FINAL: {acc_final:.4f} (levou {time.time()-t0:.2f}s)")

    # Guarda o modelo treinado nos 60k
    best_tuned_models[name] = model


--- Etapa (f): Avaliação Final no Conjunto de Teste Final (10k) ---
(Re-treinando os 3 melhores modelos em 60000 amostras)
Re-treinando e avaliando NaiveBayes otimizado...
  -> NaiveBayes | Acurácia FINAL: 0.5377 (levou 1.48s)
Re-treinando e avaliando MLP otimizado...
  -> MLP | Acurácia FINAL: 0.9703 (levou 51.49s)
Re-treinando e avaliando SGD otimizado...
  -> SGD | Acurácia FINAL: 0.9140 (levou 445.13s)


In [ ]:
# ## Célula 14: - Seleção e Exportação do Melhor Modelo
# Seleciona o melhor modelo com base na acurácia final e salva o modelo

print("\n--- Etapa (g): Seleção e Exportação do Melhor Modelo ---")

if not final_results:
    print("ERRO: Nenhum resultado final foi gerado. Não é possível exportar.")
else:
    # Cria uma Series para facilitar a ordenação
    final_results_series = pd.Series(final_results).sort_values(ascending=False)

    print("Ranking Final (Pós-Otimização, Teste Final):")
    print(final_results_series)

    # Pega o nome e a acurácia do melhor
    best_overall_name = final_results_series.idxmax()
    best_overall_accuracy = final_results_series.max()

    # Pega o objeto do modelo (já treinado nos 60k)
    model_to_export = best_tuned_models[best_overall_name]

    print(f"\nMelhor modelo geral: {best_overall_name} (Acurácia: {best_overall_accuracy:.4f})")

    scaler_filename = 'mnist_scaler.joblib'
    joblib.dump(scaler, scaler_filename)
    joblib.dump(model_to_export, MODEL_FILENAME)

    print(f"\nModelo salvo com sucesso em: {MODEL_FILENAME}")
    print(f"Scaler salvo com sucesso em: {scaler_filename}")

print(f"\n--- Pipeline Concluído ---")
print(f"Tempo total de execução: {(time.time() - start_time) / 60:.2f} minutos")


--- Etapa (g): Seleção e Exportação do Melhor Modelo ---
Ranking Final (Pós-Otimização, Teste Final):
MLP           0.9703
SGD           0.9140
NaiveBayes    0.5377
dtype: float64

Melhor modelo geral: MLP (Acurácia: 0.9703)

Modelo salvo com sucesso em: best_mnist_model.joblib
Scaler salvo com sucesso em: mnist_scaler.joblib

--- Pipeline Concluído ---
Tempo total de execução: 145.17 minutos
